In [22]:
import pandas as pd
from pathlib import Path
from glob import glob
import re

In [23]:
BFSS_WITH_DEPS = '../assets/synthesis-aiger/bfss-with-deps'
BFSS_SKIP_DEPS = '../assets/synthesis-aiger/bfss-skip-deps'

# Results File Reader

In [24]:
def read_result_file(filepath: str):
    file_results = Path(filepath).read_text()
    
    total_duration_rgx = "total duration:\s*(\d+[.]?\d*)"
    tool_name_rgx = "Tool:\s*(.*)[*][/]"
    is_completed = "Independents Aiger" in file_results
    error = "\n".join(file_results.split("\n")[2:]) if not is_completed else None
    total_duration = float(re.search(total_duration_rgx, file_results).group(1)) if is_completed else None
    tool_name = re.search(tool_name_rgx, file_results).group(1).strip()
    
    is_realizable = not("UNREALIZABLE" in file_results) and is_completed
    benchmark_name = Path(filepath).stem
    has_deps =  is_completed and (not "No dependent variables found" in file_results ) and (not "Skipped finding dependencies." in file_results)
    
    if is_completed:
        independents_aiger_rgx = "Independents Aiger:\s*\naag\s*(.*)"
        independent_aig = re.search(independents_aiger_rgx, file_results).group(1).strip()
        indep_max_var, indep_inputs, indep_latches, indep_outputs, indep_gates = independent_aig.split(" ")
    else:
        indep_max_var, indep_inputs, indep_latches, indep_outputs, indep_gates = None,  None,  None, None, None
        
    if has_deps:
        dependents_aiger_rgx = "Dependents Aiger:\s*\naag\s*(.*)"
        dependent_aig = re.search(dependents_aiger_rgx, file_results).group(1).strip()
        dep_max_var, dep_inputs, dep_latches, dep_outputs, dep_gates = dependent_aig.split(" ")
    else:
        dep_max_var, dep_inputs, dep_latches, dep_outputs, dep_gates =None,  None,  None, None, None
        
    
    return {
        'is_completed': is_completed,
        'error': error,
        'total_duration': total_duration,
        'tool_name': tool_name,
        'is_realizable': is_realizable,
        'benchmark_name': benchmark_name,
        
        'indeps_aiger_max_var': indep_max_var,
        'indeps_aiger_latches': indep_latches,
        'indeps_aiger_inputs': indep_inputs,
        'indeps_aiger_outputs': indep_latches,
        'indeps_aiger_gates': indep_gates,
        
        'deps_aiger_max_var': dep_max_var,
        'deps_aiger_latches': dep_latches,
        'deps_aiger_inputs': dep_inputs,
        'deps_aiger_outputs': dep_latches,
        'deps_aiger_gates': dep_gates
    }

In [25]:
# Benchmark with deps results
read_result_file(BFSS_WITH_DEPS + '/ltl2dpa11.hoa')

{'is_completed': True,
 'error': None,
 'total_duration': 6.0,
 'tool_name': 'bfss-synt',
 'is_realizable': True,
 'benchmark_name': 'ltl2dpa11',
 'indeps_aiger_max_var': '25',
 'indeps_aiger_latches': '2',
 'indeps_aiger_inputs': '3',
 'indeps_aiger_outputs': '2',
 'indeps_aiger_gates': '20',
 'deps_aiger_max_var': '121',
 'deps_aiger_latches': '11',
 'deps_aiger_inputs': '4',
 'deps_aiger_outputs': '11',
 'deps_aiger_gates': '106'}

In [26]:
# Unrealizable benchmark
read_result_file(BFSS_WITH_DEPS + '/OneCounterGuiA6.hoa')

{'is_completed': False,
 'error': 'UNREALIZABLE\n',
 'total_duration': None,
 'tool_name': 'bfss-synt',
 'is_realizable': False,
 'benchmark_name': 'OneCounterGuiA6',
 'indeps_aiger_max_var': None,
 'indeps_aiger_latches': None,
 'indeps_aiger_inputs': None,
 'indeps_aiger_outputs': None,
 'indeps_aiger_gates': None,
 'deps_aiger_max_var': None,
 'deps_aiger_latches': None,
 'deps_aiger_inputs': None,
 'deps_aiger_outputs': None,
 'deps_aiger_gates': None}

In [27]:
# Bennchmark with errors
read_result_file(BFSS_WITH_DEPS + '/13.hoa')

{'is_completed': False,
 'error': "terminate called after throwing an instance of 'std::runtime_error'\n  what():  Too many acceptance sets used.  The limit is 32.\n",
 'total_duration': None,
 'tool_name': 'bfss-synt',
 'is_realizable': False,
 'benchmark_name': '13',
 'indeps_aiger_max_var': None,
 'indeps_aiger_latches': None,
 'indeps_aiger_inputs': None,
 'indeps_aiger_outputs': None,
 'indeps_aiger_gates': None,
 'deps_aiger_max_var': None,
 'deps_aiger_latches': None,
 'deps_aiger_inputs': None,
 'deps_aiger_outputs': None,
 'deps_aiger_gates': None}

In [28]:
# Bennchmark incompleted
read_result_file(BFSS_WITH_DEPS + '/KitchenTimerV6.hoa')

{'is_completed': False,
 'error': '',
 'total_duration': None,
 'tool_name': 'bfss-synt',
 'is_realizable': False,
 'benchmark_name': 'KitchenTimerV6',
 'indeps_aiger_max_var': None,
 'indeps_aiger_latches': None,
 'indeps_aiger_inputs': None,
 'indeps_aiger_outputs': None,
 'indeps_aiger_gates': None,
 'deps_aiger_max_var': None,
 'deps_aiger_latches': None,
 'deps_aiger_inputs': None,
 'deps_aiger_outputs': None,
 'deps_aiger_gates': None}

In [29]:
# Bennchmark without deps
read_result_file(BFSS_WITH_DEPS + '/lilydemo13.hoa')

{'is_completed': True,
 'error': None,
 'total_duration': 1.0,
 'tool_name': 'bfss-synt',
 'is_realizable': True,
 'benchmark_name': 'lilydemo13',
 'indeps_aiger_max_var': '1',
 'indeps_aiger_latches': '0',
 'indeps_aiger_inputs': '1',
 'indeps_aiger_outputs': '0',
 'indeps_aiger_gates': '0',
 'deps_aiger_max_var': None,
 'deps_aiger_latches': None,
 'deps_aiger_inputs': None,
 'deps_aiger_outputs': None,
 'deps_aiger_gates': None}

In [30]:
benchmarks_summary = pd.read_csv('../assets/find_deps_by_automaton_summary.csv', index_col=0)
benchmarks_summary = benchmarks_summary[["name", "total_inputs", "total_outputs", "total_dependent_vars", "dependency_ratio"]]
benchmarks_summary

,name,total_inputs,total_outputs,total_dependent_vars,dependency_ratio
0,ltl2dpa20,3,2,1,0.500000
1,TwoCountersInRangeA2,11,8,3,0.375000
2,lilydemo24,4,4,0,0.000000
3,OneCounterGuiA9,9,9,3,0.333333
4,ltl2dba24,1,1,0,0.000000
...,...,...,...,...,...
193,KitchenTimerV8,4,11,0,0.000000
194,ltl2dba27,1,1,0,0.000000
195,TorcsGearing,1,3,1,0.333333
196,ltl2dpa19,3,3,1,0.333333


# Joining All Results

In [31]:
bfss_with_deps_files = glob(BFSS_WITH_DEPS + "/*")
bfss_skip_deps_files = glob(BFSS_SKIP_DEPS + "/*")

synt_results = pd.DataFrame([
    read_result_file(file)
    for file in [*bfss_with_deps_files, *bfss_skip_deps_files]
])
# synt_results.to_csv('../assets/synthesis-aiger/synt_results.csv', index=False)
synt_results

,is_completed,error,total_duration,tool_name,is_realizable,benchmark_name,indeps_aiger_max_var,indeps_aiger_latches,indeps_aiger_inputs,indeps_aiger_outputs,indeps_aiger_gates,deps_aiger_max_var,deps_aiger_latches,deps_aiger_inputs,deps_aiger_outputs,deps_aiger_gates
0,False,UNREALIZABLE\n,NaN,bfss-synt,False,TwoCountersInRangeA4,None,None,None,None,None,None,None,None,None,None
1,True,None,6.0,bfss-synt,True,ltl2dba05,143,4,4,4,135,None,None,None,None,None
2,True,None,2.0,bfss-synt,True,ltl2dba11,12,2,2,2,8,None,None,None,None,None
3,False,UNREALIZABLE\n,NaN,bfss-synt,False,TwoCountersDisButA6,None,None,None,None,None,None,None,None,None,None
4,True,None,4.0,bfss-synt,True,TorcsSteeringSimple,14,0,4,0,10,93,4,6,4,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,False,UNREALIZABLE\n,NaN,bfss-synt-skip-deps,False,OneCounterGuiA1,None,None,None,None,None,None,None,None,None,None
410,True,None,10.0,bfss-synt-skip-deps,True,EscalatorSmart,273,4,4,4,265,None,None,None,None,None
411,False,UNREALIZABLE\n,NaN,bfss-synt-skip-deps,False,OneCounterGuiA0,None,None,None,None,None,None,None,None,None,None
412,True,None,9.0,bfss-synt-skip-deps,True,ltl2dba20,189,4,4,4,181,None,None,None,None,None


In [36]:
X = synt_results.copy()
X.loc[X["tool_name"] == "bfss-synt", "tool_name"] = "Deps-Synt"
X.loc[X["tool_name"] == "bfss-synt-skip-deps", "tool_name"] = "Spot-Hijack"
X = X.pivot(index='benchmark_name', columns='tool_name', values=['total_duration', 'is_realizable', 'is_completed', 'error'])
X.columns = X.columns.to_series().str.join('_')
compared_df = pd.merge(
    X,
    benchmarks_summary,
    how='left',
    left_on='benchmark_name',
    right_on='name'
)
compared_df

,total_duration_Deps-Synt,total_duration_Spot-Hijack,is_realizable_Deps-Synt,is_realizable_Spot-Hijack,is_completed_Deps-Synt,is_completed_Spot-Hijack,error_Deps-Synt,error_Spot-Hijack,name,total_inputs,total_outputs,total_dependent_vars,dependency_ratio
0,9.0,9.0,True,True,True,True,None,None,01,4.0,3.0,0.0,0.000000
1,110.0,81.0,True,True,True,True,None,None,02,4.0,4.0,0.0,0.000000
2,5659.0,2841.0,True,True,True,True,None,None,03,5.0,5.0,0.0,0.000000
3,NaN,1598039.0,False,True,False,True,,None,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,False,False,False,False,terminate called after throwing an instance of...,terminate called after throwing an instance of...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,10.0,12.0,True,True,True,True,None,None,ltl2dpa20,3.0,2.0,1.0,0.500000
203,17.0,18.0,True,True,True,True,None,None,ltl2dpa21,2.0,3.0,1.0,0.333333
204,171444.0,223729.0,True,True,True,True,None,None,ltl2dpa22,6.0,3.0,1.0,0.333333
205,7.0,8.0,True,True,True,True,None,None,ltl2dpa23,2.0,2.0,1.0,0.500000


In [37]:
compared_df.to_csv('./aiger-synt-summary.csv')